In [1]:
import os 
import time
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

## 1 模型定义

In [2]:
class VGGBlock(nn.Module):
    def __init__(self,in_channels,out_channels,batch_norm):#在后来改良后的VGG网络增加了BatchNorm
        super(VGGBlock,self).__init__()
        stack=[]
        stack.append(nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1))
        if batch_norm:
            stack.append(nn.BatchNorm2d(out_channels))
        stack.append(nn.ReLU(inplace=True))
        self.model_block=nn.Sequential(*stack)
        
    def forward(self,x):
        return self.model_block(x)
    

class VGGNet11(nn.Module):
    def __init__(self,block,pool,batch_norm):#block是一个网络模组抽象，pool也是pooling层的抽象
        super(VGGNet11,self).__init__()
        self.feature_block=nn.Sequential(
            block(1,64,batch_norm), #32*32
            pool(kernel_size=2,stride=2),#16*16
            block(64,128,batch_norm),
            pool(kernel_size=2,stride=2),#8*8
            block(128,256,batch_norm),
            block(256,256,batch_norm),
            pool(kernel_size=2,stride=2),#4*4
            block(256,512,batch_norm),
            block(512,512,batch_norm),
            pool(kernel_size=2,stride=2),#2*2
            block(512,512,batch_norm),
            block(512,512,batch_norm),
            pool(kernel_size=2,stride=2),#1*1
        )
        self.classifier=nn.Linear(512,10)
        
    def forward(self,x):
        x=self.feature_block(x)
        x=x.view(x.shape[0],-1)
        x=self.classifier(x)
        return x

    
class VGGNet16(nn.Module):
    def __init__(self,block,pool,batch_norm): #block是一个网络模组抽象，pool也是pooling层的抽象
        super(VGGNet16,self).__init__()
        self.feature_block=nn.Sequential(
            block(1,64,batch_norm), #32*32
            block(64,64,batch_norm), #32*32
            pool(kernel_size=2,stride=2),#16*16
            block(64,128,batch_norm),
            block(128,128,batch_norm),
            pool(kernel_size=2,stride=2),#8*8
            block(128,256,batch_norm),
            block(256,256,batch_norm),
            pool(kernel_size=2,stride=2),#4*4
            block(256,512,batch_norm),
            block(512,512,batch_norm),
            block(512,512,batch_norm),
            pool(kernel_size=2,stride=2),#2*2
            block(512,512,batch_norm),
            block(512,512,batch_norm),
            block(512,512,batch_norm),
            pool(kernel_size=2,stride=2),#1*1
        )
        self.classifier=nn.Linear(512,10)
        
    def forward(self,x):
        x=self.feature_block(x)
        x=x.view(x.shape[0],-1)
        x=self.classifier(x)
        return x

## 2 模型训练与评估类

In [3]:
def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        func(*args,**kwargs)
        end = time.time()
        cost = end - start
        print("Cost time: {} mins.".format(cost/60)) 
    return wrapper


class CNNModel(object):
    def __init__(self, model, train_data, test_data, model_dir, model_name,
                 best_valid_loss=float('inf'), n_split=0.9, batch_size=64, epochs=10):
        self.batch_size = batch_size
        self.epochs = epochs
        self.best_valid_loss = best_valid_loss
        self.model_dir = model_dir
        self.model_name = model_name
        self.n_split = n_split
        
        self.train_data =  train_data
        self.test_data = test_data
        
        self.device = self.get_device()
        self.init_data()
        self.init_iterator()
        self.init_model_path()
        
        self.model = self.init_model(model)
        self.optimizer = self.set_optimizer()
        self.criterion = self.set_criterion()
        
    def get_device(self):
        d = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        return d
    
    def init_data(self):
        n_train = int(len(self.train_data)*self.n_split)
        n_validation = len(self.train_data) - n_train
        self.train_data, self.valid_data = torch.utils.data.random_split(self.train_data, [n_train, n_validation])
    
    def init_iterator(self):
        self.train_iterator = torch.utils.data.DataLoader(self.train_data, shuffle=True, batch_size=self.batch_size)
        self.valid_iterator = torch.utils.data.DataLoader(self.valid_data, batch_size=self.batch_size)
        self.test_iterator = torch.utils.data.DataLoader(self.test_data, batch_size=self.batch_size)
        
    def set_optimizer(self):
        optimizer = optim.Adam(self.model.parameters()) 
        return optimizer
    
    def set_criterion(self):
        criterion = nn.CrossEntropyLoss()
        return criterion
    
    def init_model(self, model):
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
        model = model.to(self.device)
        return model
        
    def init_model_path(self):
        if not os.path.isdir(self.model_dir):
            os.makedirs(self.model_dir)
        self.model_path = os.path.join(self.model_dir, self.model_name)
        
    # 定义评估函数
    def accu(self, fx, y):
        pred = fx.max(1,keepdim=True)[1]
        correct = pred.eq(y.view_as(pred)).sum()  # 得到该batch的准确度
        acc = correct.float()/pred.shape[0]
        return acc

    def train(self):
        epoch_loss = 0   # 积累变量
        epoch_acc = 0    # 积累变量
        self.model.train()    # 该函数表示PHASE=Train

        for (x,y) in self.train_iterator:  # 拿去每一个minibatch
            x = x.to(self.device)
            y = y.to(self.device)
            self.optimizer.zero_grad()
            fx = self.model(x)           # 进行forward
            loss = self.criterion(fx,y)  # 计算Loss,train_loss
            type(loss)
            acc = self.accu(fx,y)      # 计算精确度，train_accu
            loss.backward()     # 进行BP
            self.optimizer.step()    # 统一更新模型
            epoch_loss += loss.item()
            epoch_acc += acc.item()

        return epoch_loss/len(self.train_iterator),epoch_acc/len(self.train_iterator)

    def evaluate(self, iterator):
        epoch_loss = 0
        epoch_acc = 0
        self.model.eval()
        with torch.no_grad():
            for (x,y) in iterator:
                x = x.to(self.device)
                y = y.to(self.device)
                fx = self.model(x)
                loss = self.criterion(fx,y)
                acc = self.accu(fx,y)
                epoch_loss += loss.item()
                epoch_acc += acc.item()
        return epoch_loss/len(iterator),epoch_acc/len(iterator)
    
    @timer
    def train_fit(self):
        info = 'Epoch:{0} | Train Loss:{1} | Train Acc:{2} | Val Loss:{3} | Val Acc:{4}'
        for epoch in range(self.epochs):
            train_loss, train_acc = self.train()
            valid_loss, valid_acc = self.evaluate(self.valid_iterator)
            if valid_loss < self.best_valid_loss:  # 如果是最好的模型就保存到文件夹
                self.best_valid_loss = valid_loss
                torch.save(self.model.state_dict(), self.model_path)
            print(info.format(epoch+1, train_loss, train_acc, valid_loss, valid_acc))
    
    def get_acc(self):
        self.model.load_state_dict(torch.load(self.model_path))
        test_loss, test_acc = self.evaluate(self.test_iterator)
        print('| Test Loss: {0} | Test Acc: {1} |'.format(test_loss,test_acc))


## 3 数据集的准备

In [4]:
data_trans = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [5]:
train_data = datasets.MNIST('data', train=True, download=True, transform=data_trans)
test_data = datasets.MNIST('data', train=False, download=True, transform=data_trans)

## 4 模型训练

### 4.1 VGG11

In [6]:
epochs = 1
n_split = 0.9
batch_size = 64
model_dir = 'models'
best_valid_loss = float('inf')

model_name = "vgg11_mnist.pt"
model = VGGNet11(VGGBlock, nn.MaxPool2d, True)

vgg11 = CNNModel(model=model, 
               train_data=train_data, 
               test_data=test_data, 
               model_dir=model_dir, 
               model_name=model_name,
               best_valid_loss=best_valid_loss, 
               n_split=n_split, 
               batch_size=batch_size, 
               epochs=epochs)

In [7]:
print(vgg11.device)

cuda


In [8]:
print(vgg11.model)

DataParallel(
  (module): VGGNet11(
    (feature_block): Sequential(
      (0): VGGBlock(
        (model_block): Sequential(
          (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace)
        )
      )
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): VGGBlock(
        (model_block): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace)
        )
      )
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): VGGBlock(
        (model_block): Sequential(
          (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affi

In [9]:
vgg11.train_fit()

Epoch:1 | Train Loss:0.12352728955138705 | Train Acc:0.962918394549763 | Val Loss:0.06760358097071344 | Val Acc:0.9780585106382979
Cost time: 3.2129016399383543 mins.


In [10]:
vgg11.get_acc()

| Test Loss: 0.06335935833015639 | Test Acc: 0.9810907643312102 |


### 4.2 VGG16

In [11]:
epochs = 1
n_split = 0.9
batch_size = 64
model_dir = 'models'
best_valid_loss = float('inf')

model_name = "vgg16_mnist.pt"
model = VGGNet16(VGGBlock, nn.MaxPool2d, True)

vgg16 = CNNModel(model=model, 
               train_data=train_data, 
               test_data=test_data, 
               model_dir=model_dir, 
               model_name=model_name,
               best_valid_loss=best_valid_loss, 
               n_split=n_split, 
               batch_size=batch_size, 
               epochs=epochs)

In [12]:
vgg16.model

DataParallel(
  (module): VGGNet16(
    (feature_block): Sequential(
      (0): VGGBlock(
        (model_block): Sequential(
          (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace)
        )
      )
      (1): VGGBlock(
        (model_block): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace)
        )
      )
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): VGGBlock(
        (model_block): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace)
        )
      )
      (

In [13]:
vgg16.train_fit()

Epoch:1 | Train Loss:0.18191335635851202 | Train Acc:0.9438684834123223 | Val Loss:0.05115903031199853 | Val Acc:0.9867021276595744
Cost time: 4.865456604957581 mins.


In [14]:
vgg16.get_acc()

| Test Loss: 0.04106367449424449 | Test Acc: 0.9876592356687898 |
